Lab 2

MSSV: 19110392

Họ tên: Trần Quang Nghĩa

Môn: Máy học nâng cao

In [2]:
#import 
import keras
from keras import backend as k
import tensorflow as tf
import numpy as np

In [3]:
# lamda
regularizer=0.0005

In [5]:
# CIFAR10 dataset
from keras.datasets import cifar10

(X_train,y_train),(X_test,y_test) = cifar10.load_data()
print("Kích thước của X_train là: ",X_train.shape)
print("Kích thước của y_train là: ",y_train.shape)
print("Kích thước của X_test là: ",X_test.shape)
print("Kích thước của y_test là: ",y_test.shape)

Kích thước của X_train là:  (50000, 32, 32, 3)
Kích thước của y_train là:  (50000, 1)
Kích thước của X_test là:  (10000, 32, 32, 3)
Kích thước của y_test là:  (10000, 1)


In [7]:
import cv2
def resize_img(img):
    numberOfImage = img.shape[0]
    new_array = np.zeros((numberOfImage, 48,48,3))
    for i in range(numberOfImage):
        new_array[i] = cv2.resize(img[i,:,:,:],(48,48))
    return new_array

In [8]:
X_train = resize_img(X_train)
X_test = resize_img(X_test)
print("Kích thước mới của X_train là: ",X_train.shape)
print("Kích thước mới của X_test là: ",X_test.shape)

Kích thước mới của X_train là:  (50000, 48, 48, 3)
Kích thước mới của X_test là:  (10000, 48, 48, 3)


In [9]:
# One hot label encoding
from keras.utils import to_categorical
y_train = to_categorical(y_train,num_classes=10)
y_test = to_categorical(y_test,num_classes=10)

print("Kích thước mới của y_train là: ",y_train.shape)
print("Kích thước mới của y_test là: ",y_test.shape)

Kích thước mới của y_train là:  (50000, 10)
Kích thước mới của y_test là:  (10000, 10)


In [10]:
# VGG19 Model 
from keras.applications.vgg19 import VGG19

# Include top = add fully connected layers to layer.
# Weights = use pretrained weights (trained in imagenet)
vgg = VGG19(include_top=False,weights="imagenet",input_shape=(48,48,3))

vgg.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 48, 48, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 48, 48, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 48, 48, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 24, 24, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 24, 24, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 24, 24, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 12, 12, 128)       0     

In [11]:
model = keras.Sequential()

# Adding layers to the blank model
for layer in vgg.layers:
    model.add(layer)
    
# Don't train layers again, because they are already trained
for layer in model.layers:
    layer.trainable = False
    
# Adding fully connected layers
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128))
model.add(keras.layers.Dense(10,activation="softmax"))

# Checking model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 48, 48, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 48, 48, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 24, 24, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 24, 24, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 24, 24, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 12, 12, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 12, 12, 256)       2

In [17]:
#compile the model
model.compile(optimizer="RMSprop",loss="categorical_crossentropy",metrics=["accuracy"])

# train.
hist = model.fit(X_train,y_train,validation_split=0.15,epochs=5,batch_size=200)

# We use %15 of the train set as validation set.

Epoch 1/5
213/213 [==============================] - 1471s 7s/step - loss: 1.8765 - accuracy: 0.5442 - val_loss: 1.9432 - val_accuracy: 0.5036
Epoch 2/5
213/213 [==============================] - 1463s 7s/step - loss: 1.5484 - accuracy: 0.5577 - val_loss: 1.5370 - val_accuracy: 0.5424
Epoch 3/5
213/213 [==============================] - 1467s 7s/step - loss: 1.3610 - accuracy: 0.5705 - val_loss: 1.3413 - val_accuracy: 0.5700
Epoch 4/5
213/213 [==============================] - 1461s 7s/step - loss: 1.2512 - accuracy: 0.5842 - val_loss: 1.3003 - val_accuracy: 0.5747
Epoch 5/5
213/213 [==============================] - 1462s 7s/step - loss: 1.1953 - accuracy: 0.5942 - val_loss: 1.2797 - val_accuracy: 0.5753


In [18]:
model.evaluate(X_test,y_test)

313/313 [==============================] - 299s 956ms/step - loss: 1.2782 - accuracy: 0.5714


[1.2782456874847412, 0.571399986743927]



*   epoch: số lần train. Khi epoch tăng (giảm) thì số lần train sẽ tăng (giảm).
*   batch_size: số lượng mẫu mỗi lần cập nhật gradient
